In [7]:
class FSA:
    def __init__(self):
        self.transitions = {}
        self.start = 0
        self.finals = set()

    def add_transition(self, s, c, n):
        self.transitions.setdefault(s, {})[c] = n

    def add_final(self, s):
        self.finals.add(s)

    def accepts(self, word):
        s = self.start
        for c in word:
            if s not in self.transitions or c not in self.transitions[s]:
                return False
            s = self.transitions[s][c]
        return s in self.finals


def build_noun_fsa():
    f = FSA()
    state = 0

    for root in ["cat", "dog", "fox"]:
        s = state
        for c in root:
            ns = state + 1
            f.add_transition(s, c, ns)
            s = ns
            state = ns

        f.add_final(s)

        if root == "fox":
            f.add_transition(s, 'e', state + 1)
            f.add_transition(state + 1, 's', state + 2)
            f.add_final(state + 2)
            state += 2
        else:
            f.add_transition(s, 's', state + 1)
            f.add_final(state + 1)
            state += 1

    return f


def build_verb_fsa():
    f = FSA()
    state = 0

    for root in ["walk", "jump"]:
        s = state
        for c in root:
            ns = state + 1
            f.add_transition(s, c, ns)
            s = ns
            state = ns

        f.add_final(s)

        f.add_transition(s, 's', state + 1)
        f.add_final(state + 1)

        f.add_transition(s, 'e', state + 2)
        f.add_transition(state + 2, 'd', state + 3)
        f.add_final(state + 3)

        f.add_transition(s, 'i', state + 4)
        f.add_transition(state + 4, 'n', state + 5)
        f.add_transition(state + 5, 'g', state + 6)
        f.add_final(state + 6)

        state += 6

    return f


def stem_suffix(word):
    if word.endswith("ing"):
        return word[:-3], "ing"
    if word.endswith("ed"):
        return word[:-2], "ed"
    if word.endswith("es"):
        return word[:-2], "es"
    if word.endswith("s"):
        return word[:-1], "s"
    return word, "NULL"


noun_fsa = build_noun_fsa()
verb_fsa = build_verb_fsa()


print("\nQ1: Singular and Plural Nouns Recognition")
nouns = ["cat", "cats", "dog", "dogs", "fox", "foxes"]
for w in nouns:
    print(w, "→", "ACCEPTED" if noun_fsa.accepts(w) else "REJECTED")


print("\nQ2 & Q3: Verb Inflections Recognition")
verbs = ["walk", "walks", "walking", "walked",
         "jump", "jumps", "jumping", "jumped"]
for w in verbs:
    print(w, "→", "ACCEPTED" if verb_fsa.accepts(w) else "REJECTED")


print("\nQ4: ACCEPTED / REJECTED Test Words")
test_words = ["cats", "foxes", "foxs", "walking", "walkes"]
for w in test_words:
    print(w, "→", "ACCEPTED" if noun_fsa.accepts(w) or verb_fsa.accepts(w) else "REJECTED")


print("\nQ5 & Q8: Morphological Parsing (word → stem + suffix)")
for w in test_words:
    if noun_fsa.accepts(w) or verb_fsa.accepts(w):
        stem, suf = stem_suffix(w)
        print(w, "→", stem, "+", suf)


print("\nQ6, Q7, Q9 & Q10: Final Classification")
all_words = nouns + verbs + test_words
for w in all_words:
    if noun_fsa.accepts(w):
        stem, suf = stem_suffix(w)
        print(w, "→ NOUN →", stem, "+", suf)
    elif verb_fsa.accepts(w):
        stem, suf = stem_suffix(w)
        print(w, "→ VERB →", stem, "+", suf)
    else:
        print(w, "→ INVALID MORPHOLOGICAL FORM")



Q1: Singular and Plural Nouns Recognition
cat → ACCEPTED
cats → ACCEPTED
dog → REJECTED
dogs → REJECTED
fox → REJECTED
foxes → REJECTED

Q2 & Q3: Verb Inflections Recognition
walk → ACCEPTED
walks → ACCEPTED
walking → ACCEPTED
walked → ACCEPTED
jump → REJECTED
jumps → REJECTED
jumping → REJECTED
jumped → REJECTED

Q4: ACCEPTED / REJECTED Test Words
cats → ACCEPTED
foxes → REJECTED
foxs → REJECTED
walking → ACCEPTED
walkes → REJECTED

Q5 & Q8: Morphological Parsing (word → stem + suffix)
cats → cat + s
walking → walk + ing

Q6, Q7, Q9 & Q10: Final Classification
cat → NOUN → cat + NULL
cats → NOUN → cat + s
dog → INVALID MORPHOLOGICAL FORM
dogs → INVALID MORPHOLOGICAL FORM
fox → INVALID MORPHOLOGICAL FORM
foxes → INVALID MORPHOLOGICAL FORM
walk → VERB → walk + NULL
walks → VERB → walk + s
walking → VERB → walk + ing
walked → VERB → walk + ed
jump → INVALID MORPHOLOGICAL FORM
jumps → INVALID MORPHOLOGICAL FORM
jumping → INVALID MORPHOLOGICAL FORM
jumped → INVALID MORPHOLOGICAL FORM
cats